# Important importations

In [ ]:
import time
import matplotlib.pyplot as plt
import numpy as np

from matplotlib.pyplot import gca
from mpl_toolkits.axes_grid1 import make_axes_locatable

from scipy.optimize import brentq
from scipy.interpolate import interp1d
from scipy.stats import norm

import astropy.units as u
from astropy.coordinates import SkyCoord
from astropy.units import Quantity
from astropy.io import fits
from regions import CircleSkyRegion

from gammapy.irf import load_cta_irfs
from gammapy.maps import WcsGeom, MapAxis, WcsNDMap
from gammapy.modeling.models import (PowerLawSpectralModel, 
                                     TemplateSpatialModel, 
                                     PointSpatialModel, 
                                     SkyModel, 
                                     Models, 
                                     FoVBackgroundModel)

from gammapy.makers import MapDatasetMaker
from gammapy.modeling import Fit
from gammapy.data import Observation
from gammapy.datasets import MapDataset, FluxPointsDataset
from gammapy.estimators import FluxPointsEstimator

from gammapy.astro.darkmatter import (profiles, 
                                      JFactory, 
                                      PrimaryFlux, 
                                      DarkMatterAnnihilationSpectralModel)

In [ ]:
#%env GAMMAPY_DATA=/your_path/cta_school/

# Exercise 1: Simulating a DM signal in the galactic centre

## Create the dataset

In [ ]:
# Define basic simulation parameters
livetime = 520.0 * u.hr
GLON = 0.0 * u.deg
GLAT = 0.0 * u.deg
pointing = SkyCoord(0.0, 0.0, unit="deg", frame="galactic")

emin = 20/1000
emax = 150

energy_axes = MapAxis.from_energy_bounds(emin, emax, nbin=10, unit="TeV", name="energy")
region = CircleSkyRegion(center=pointing, radius=10.0 * u.deg)

geom = WcsGeom.create(
    skydir=pointing,
    binsz=0.02,
    width=(20, 20),
    frame="galactic",
    axes=[energy_axes],
)

empty = MapDataset.create(geom, name="dataset-simu")

The expected gamma-ray flux for the case in which the DM is annihilating is defined as:

$\frac{d\Phi_\gamma}{dE}=scale \times J_{factor}\;\times \dfrac{\langle \sigma v \rangle}{4 \pi \; m_{\rm DM}^2} \sum_i {\rm BR}_i \dfrac{dN_\gamma^{\rm i}}{dE}$

where:

- $scale=1$ (default), this will be the fit variable

- $J_{factor}$ is the astrophysical factor

- $m_{DM}$ is the DM particle mass

- $i$ is the selected annihilation channel

- $\langle \sigma v \rangle= 3\times 10^{-26}$ cm$^3$ s$^{-1}$ (default)

- $BR=1$ (default)

In [ ]:
# **Define DM model**

# Spectral
JFAC = 5.5e+20 * u.Unit("GeV2 cm-5") # Galactic centre [2007.16129] Einasto 10 deg
mDM = 5000*u.Unit("GeV")
channel = "b"
redshift = 0.0 # For extragalactic objects, we can add their redshift and also the EBL effect
scale = 100 # As we want to make sure to have a signal
dm_model = DarkMatterAnnihilationSpectralModel(
    mass=mDM, 
    channel=channel, 
    jfactor=JFAC, 
    z=redshift,
    scale=scale
)

# Plot spectral model to simulate
fig_1 = plt.figure()
plt.plot()
dm_model.plot([emin*u.TeV, emax*u.TeV], energy_power=0)
form = plt.FormatStrFormatter('$%g$')
gca().xaxis.set_major_formatter(form)
plt.show()

In [ ]:
# Spatial - We read the emission template created outside gammapy
jfactor_filename = '/your_path/cta_school/annihil_gal2D_gNFW_gamma_1.26_Rodot_8.3kpc_rhoodot_0.4_LOS0_0_allsky_nside1024-JFACTOR_PER_SR-Jtot_per_sr-image-image.fits'
hdul = fits.open(jfactor_filename)
dm_spatial_model = TemplateSpatialModel.read(jfactor_filename)

In [ ]:
# Set the sky model 
model_simu = SkyModel(spatial_model=dm_spatial_model, spectral_model=dm_model, name="gc")
print(model_simu)

In [ ]:
# **Define BKG model**
bkg_model = FoVBackgroundModel(dataset_name="dataset-simu")
print(bkg_model)

In [ ]:
# Gather all the models
models = Models([model_simu, bkg_model])
print(models)

In [ ]:
# Load IRFs
irfs = load_cta_irfs(
    "$GAMMAPY_DATA/Prod5-South-20deg-AverageAz-14MSTs37SSTs.180000s-v0.1.fits"
)

In [ ]:
# Create the observation
obs = Observation.create(pointing=pointing, livetime=livetime, irfs=irfs)
print(obs)

In [ ]:
# Make the MapDataset
maker = MapDatasetMaker(selection=["exposure", "background", "psf", "edisp"])
dataset = maker.run(empty, obs)
dataset.models = models
dataset.fake(random_state=int(time.time())) # Here is where we combine models+IRFs and create a Poisson realization

In [ ]:
# Save the created dataset
dataset.write("/your_path/cta_school/dataset-gc_1.fits", overwrite=True)

## Inspect created dataset

In [ ]:
# Let's make some plots!

fig_peek, axs = plt.subplots(2,2, figsize=(7,7))

img_1 = axs[0,0].imshow(
    np.sum(dataset.counts.data, axis=0),
    extent=(10.0+0.0,-10.0+0.0,10.0+0.0,-10.0+0.0),
    origin = 'lower',
    cmap='YlOrBr',
)
axs[0,0].set_title('Counts')
divider = make_axes_locatable(axs[0,0])
cax = divider.append_axes('right', size='5%', pad=0.05)
cbar_1 = fig_peek.colorbar(img_1, cax=cax, orientation='vertical')

img_2 = axs[0,1].imshow(
    np.sum(dataset.background.data, axis=0),
    extent=(10.0+0.0,-10.0+0.0,10.0+0.0,-10.0+0.0),
    origin = 'lower',
    cmap='YlOrBr'
)
axs[0,1].set_title('Background')
divider = make_axes_locatable(axs[0,1])
cax = divider.append_axes('right', size='5%', pad=0.05)
cbar_2 = fig_peek.colorbar(img_2, cax=cax, orientation='vertical')

img_3 = axs[1,0].imshow(
    np.sum(dataset.counts.data, axis=0) - np.sum(dataset.background.data, axis=0),
    extent=(10.0+0.0,-10.0+0.0,10.0+0.0,-10.0+0.0),
    origin = 'lower',
    cmap='YlOrBr'
)
axs[1,0].set_title('Excess')
divider = make_axes_locatable(axs[1,0])
cax = divider.append_axes('right', size='5%', pad=0.05)
cbar_3 = fig_peek.colorbar(img_3, cax=cax, orientation='vertical')

img_4 = axs[1,1].imshow(
    np.sum(dataset.exposure.data, axis=0),
    extent=(10.0+0.0,-10.0+0.0,10.0+0.0,-10.0+0.0),
    origin = 'lower',
    cmap='YlOrBr'
)
axs[1,1].set_title('Exposure')
divider = make_axes_locatable(axs[1,1])
cax = divider.append_axes('right', size='5%', pad=0.05)
cbar_4 = fig_peek.colorbar(img_4, cax=cax, orientation='vertical')
cbar_4.ax.set_ylabel(r'm$^2$')

fig_peek.subplots_adjust(wspace=0.45, hspace=0.2)

plt.plot()

In [ ]:
# Let's check the spectrrum and the different contributions
spec, axs = plt.subplots(1, 1, figsize=(6, 4))
dataset.counts.get_spectrum().plot(label='Total counts')
dataset.npred_background().get_spectrum().plot(label='BKG counts')
#dataset.npred_signal().get_spectrum().plot(label='DM counts')
axs.set_ylabel('Counts', fontsize=12)
axs.legend()
plt.plot()

In [ ]:
dataset.excess.plot_interactive()

# Excercise 2: Analyze first provided dataset

In [ ]:
# Read the dataset prepared
dataset_1 = MapDataset.read("/your_path/cta_school/dataset-gc_2.fits")
print(dataset_1)

## Inspect a dataset

In [ ]:
# Check properties of the dataset
print(dataset_1.geoms)

In [ ]:
# Let's make some plots!

fig_peek, axs = plt.subplots(2,2, figsize=(7,7))

img_1 = axs[0,0].imshow(
    np.sum(dataset_1.counts.data, axis=0),
    extent=(10.0+0.0,-10.0+0.0,10.0+0.0,-10.0+0.0),
    origin = 'lower',
    cmap='YlOrBr',
)
axs[0,0].set_title('Counts')
divider = make_axes_locatable(axs[0,0])
cax = divider.append_axes('right', size='5%', pad=0.05)
cbar_1 = fig_peek.colorbar(img_1, cax=cax, orientation='vertical')

img_2 = axs[0,1].imshow(
    np.sum(dataset_1.background.data, axis=0),
    extent=(10.0+0.0,-10.0+0.0,10.0+0.0,-10.0+0.0),
    origin = 'lower',
    cmap='YlOrBr'
)
axs[0,1].set_title('Background')
divider = make_axes_locatable(axs[0,1])
cax = divider.append_axes('right', size='5%', pad=0.05)
cbar_2 = fig_peek.colorbar(img_2, cax=cax, orientation='vertical')

img_3 = axs[1,0].imshow(
    np.sum(dataset_1.counts.data, axis=0) - np.sum(dataset_1.background.data, axis=0),
    extent=(10.0+0.0,-10.0+0.0,10.0+0.0,-10.0+0.0),
    origin = 'lower',
    cmap='YlOrBr'
)
axs[1,0].set_title('Excess')
divider = make_axes_locatable(axs[1,0])
cax = divider.append_axes('right', size='5%', pad=0.05)
cbar_3 = fig_peek.colorbar(img_3, cax=cax, orientation='vertical')

img_4 = axs[1,1].imshow(
    np.sum(dataset_1.exposure.data, axis=0),
    extent=(10.0+0.0,-10.0+0.0,10.0+0.0,-10.0+0.0),
    origin = 'lower',
    cmap='YlOrBr'
)
axs[1,1].set_title('Exposure')
divider = make_axes_locatable(axs[1,1])
cax = divider.append_axes('right', size='5%', pad=0.05)
cbar_4 = fig_peek.colorbar(img_4, cax=cax, orientation='vertical')
cbar_4.ax.set_ylabel(r'm$^2$')

fig_peek.subplots_adjust(wspace=0.45, hspace=0.2)

plt.plot()

Seems like we have something here, let's take a closer look!

In [ ]:
# Let's use the gammapy built-in functions to check the residuals 
spec_res_1, axs = plt.subplots(1, 1, figsize=(6, 4))
dataset_1.plot_residuals_spectral(method="diff", ax=axs)
plt.plot()

In [ ]:
spat_res_1, axs = plt.subplots(1, 1, figsize=(6, 4))
dataset_1.plot_residuals_spatial(method="diff", ax=axs)
plt.plot()

In [ ]:
spec, axs = plt.subplots(1, 1, figsize=(6, 4))
dataset_1.counts.get_spectrum().plot(label='Total counts')
dataset_1.npred_background().get_spectrum().plot(label='BKG counts')
axs.set_ylabel('Counts', fontsize=12)
axs.legend()
plt.plot()

In [ ]:
dataset_1.excess.plot_interactive()

## Guess what's going on

Before searching for an specific DM signal (mass, channel and $\langle \sigma v \rangle$), let's try a simple toy power-law point-like model because we first want an easy way to discard the background hypothesis

In [ ]:
# **Define 3D Sky Model**

# We define a power-law spectrum 
spectral_model_fit = PowerLawSpectralModel(index=2)

# Define the point-like source
spatial_model_fit = PointSpatialModel(lon_0=GLON, lat_0=GLAT, unit='deg', frame="galactic")

# Gather spectral and spatial models
sky_model_fit = SkyModel(spatial_model=spatial_model_fit, spectral_model=spectral_model_fit, name='sky_model')

# Add the background model extracted from the dataset
bkg_model_fit = FoVBackgroundModel(dataset_name=dataset_1.name)

# Combine source model + bkg model
models_fit = Models([sky_model_fit, bkg_model_fit])

# Let's fix the position of the source
models_fit.parameters["lon_0"].frozen = True
models_fit.parameters["lat_0"].frozen = True

# Model with only bkg
models_nosrc = Models([bkg_model_fit])

In [ ]:
# Fit with the model with only background
dataset_1.models = models_nosrc
fit = Fit() # default is minuit minimizer
result_fit_nosrc_1 = fit.run(datasets=[dataset_1])

In [ ]:
# Fit with the model with background + point-like source
dataset_1.models = models_fit
fit = Fit()
result_1 = fit.run(datasets=[dataset_1])

In [ ]:
# Let's perform the test likelihood ratio between both models

# Gammapy returns the -2 log(L)
logL_src = result_1.optimize_result.total_stat
logL_nosrc = result_fit_nosrc_1.optimize_result.total_stat

ts_1 = (logL_nosrc - logL_src)

print(ts_1, np.sqrt(ts_1))

In [ ]:
# We can easely see the results of the Fit in table format
result_1.parameters.to_table()

## Fit to a DM model

Clearly we have a signal! Now let's see if it is compatible with a DM signal

To make it more interesting, let's make a competition. Who obtain the highest TS for a DM model, wins!
Remember, you can change the DM mass and channel of the model to fit the dataset

In [ ]:
from re import X
### 
mass_fit = X*u.GeV
channel_fit = "X" 
        
# Let's define the DM model to fit
flux_model_fit = DarkMatterAnnihilationSpectralModel(mass=mass_fit, channel=channel_fit, jfactor=JFAC)
        
# Our source has spatial extension (read the input template for creating the simulation)
dmmodel_fit = SkyModel(spatial_model = dm_spatial_model, 
                           spectral_model = flux_model_fit)
        
# Finally we add both models (background+source) 
models_fit_dm = Models([dmmodel_fit, bkg_model_fit])

# We initialize the background parameters
models_fit_dm.parameters['norm'].value = 1
models_fit_dm.parameters['tilt'].value = 0

# Add the models to our dataset!
dataset_1.models = models_fit_dm

# Let's fit
fit = Fit()
result_dm_1 = fit.run(datasets=[dataset_1])

In [ ]:
# We can easely see the results of the Fit in table format
result_dm_1.parameters.to_table()

In [ ]:
# Let's perform the test likelihood ratio between both models

# Gammapy returns the -2 log(L)
logL_dm   = result_dm_1.optimize_result.total_stat

ts_dm = (logL_nosrc - logL_dm)

print(ts_dm, np.sqrt(ts_dm)) ### test statistic is too low, <5. 

# Excercise 2: Analyze second provided dataset

In [ ]:
# Read the provided dataset
dataset_2 = MapDataset.read("/your_path/cta_school/dataset-gc_3.fits")
print(dataset_2)

## Inspect a dataset

In [ ]:
# Check some dataset properties
print(dataset_2.geoms)

In [ ]:
# Let's make some plots!

fig_peek, axs = plt.subplots(2,2, figsize=(6,6))

img_1 = axs[0,0].imshow(
    np.sum(dataset_2.counts.data, axis=0),
    extent=(10.0+0.0,-10.0+0.0,10.0+0.0,-10.0+0.0),
    origin = 'lower',
    cmap='YlOrBr',
)
axs[0,0].set_title('Counts')
divider = make_axes_locatable(axs[0,0])
cax = divider.append_axes('right', size='5%', pad=0.05)
cbar_1 = fig_peek.colorbar(img_1, cax=cax, orientation='vertical')

img_2 = axs[0,1].imshow(
    np.sum(dataset_2.background.data, axis=0),
    extent=(10.0+0.0,-10.0+0.0,10.0+0.0,-10.0+0.0),
    origin = 'lower',
    cmap='YlOrBr'
)
axs[0,1].set_title('Background')
divider = make_axes_locatable(axs[0,1])
cax = divider.append_axes('right', size='5%', pad=0.05)
cbar_2 = fig_peek.colorbar(img_2, cax=cax, orientation='vertical')

img_3 = axs[1,0].imshow(
    np.sum(dataset_2.counts.data, axis=0) - np.sum(dataset_2.background.data, axis=0),
    extent=(10.0+0.0,-10.0+0.0,10.0+0.0,-10.0+0.0),
    origin = 'lower',
    cmap='YlOrBr'
)
axs[1,0].set_title('Excess')
divider = make_axes_locatable(axs[1,0])
cax = divider.append_axes('right', size='5%', pad=0.05)
cbar_3 = fig_peek.colorbar(img_3, cax=cax, orientation='vertical')

img_4 = axs[1,1].imshow(
    np.sum(dataset_2.exposure.data, axis=0),
    extent=(10.0+0.0,-10.0+0.0,10.0+0.0,-10.0+0.0),
    origin = 'lower',
    cmap='YlOrBr'
)
axs[1,1].set_title('Exposure')
divider = make_axes_locatable(axs[1,1])
cax = divider.append_axes('right', size='5%', pad=0.05)
cbar_4 = fig_peek.colorbar(img_4, cax=cax, orientation='vertical')
cbar_4.ax.set_ylabel(r'm$^2$')

fig_peek.subplots_adjust(wspace=0.45, hspace=0.2)

plt.plot()

In [ ]:
fig, axs = plt.subplots(1, 1, figsize=(6, 4))
dataset_2.plot_residuals_spectral(method="diff", ax=axs)
plt.plot()

In [ ]:
fig, axs = plt.subplots(1, 1, figsize=(6, 4))
dataset_2.plot_residuals_spatial(method="diff", ax=axs)
plt.plot()

In [ ]:
spec, axs = plt.subplots(1, 1, figsize=(6, 4))
dataset_2.counts.get_spectrum().plot(label='Total counts')
dataset_2.npred_background().get_spectrum().plot(label='BKG counts')
axs.set_ylabel('Counts', fontsize=12)
axs.legend()
plt.plot()

In [ ]:
dataset_2.excess.plot_interactive()

## Guess what's going on

Let's see if there is a signal on this data and we can guess what it is!
Before searching for an specific DM signal (mass, channel and $\langle \sigma v \rangle$), let's try the simple toy power-law point-like model we used before

In [ ]:
# Add the background model extracted from the dataset
bkg_model_fit = FoVBackgroundModel(dataset_name=dataset_2.name)

# Combine source model + bkg model
models_fit = Models([sky_model_fit, bkg_model_fit])

# Let's fix the position of the source
models_fit.parameters["lon_0"].frozen = True
models_fit.parameters["lat_0"].frozen = True

# Model with only bkg
models_nosrc = Models([bkg_model_fit])

In [ ]:
# Fit with the model with only background
dataset_2.models = models_nosrc
fit = Fit() # default is minuit minimizer
result_fit_nosrc_2 = fit.run(datasets=[dataset_2])

In [ ]:
# Fit with the model with background + point-like source
dataset_2.models = models_fit
fit = Fit()
result_2 = fit.run(datasets=[dataset_2])

In [ ]:
# Let's perform the test likelihood ratio between both models

# Gammapy returns the -2 log(L)
logL_src   = result_2.optimize_result.total_stat
logL_nosrc = result_fit_nosrc_2.optimize_result.total_stat

ts_2 = (logL_nosrc - logL_src)

print(ts_2, np.sqrt(ts_2))

The likelihood test values tell us that there is no preference for the detection model.

In [ ]:
# We can easely see the results of the Fit in table format
result_2.parameters.to_table()

## Fit to a DM model 

 
Since $\sqrt{(TS)}<5$ we have no positive detection, we shall produce an 95% CL upper limits to "scale" parameter by solving the equation:

TS (scale) = 2.71
    
Then we can transform the limits on $scale$ to $\rightarrow$ $\langle \sigma v \rangle$ limits


In [ ]:
### 
mass_fit = 5000*u.GeV
channel_fit = "b"
        
# Let's define the source model
flux_model_fit = DarkMatterAnnihilationSpectralModel(mass=mass_fit, channel=channel_fit, jfactor=JFAC)
        
# Our source has spatial extension (read the input template for creating the simulation)
dmmodel_fit = SkyModel(spatial_model = dm_spatial_model, 
                           spectral_model = flux_model_fit)
        
# Finally we add both models (background+source) 
models_fit_dm = Models([dmmodel_fit, bkg_model_fit])

# WE initialize the background parameters
models_fit_dm.parameters['norm'].value = 1
models_fit_dm.parameters['tilt'].value = 0

# Add the models to our dataset!
dataset_2.models = models_fit_dm

# Let's fit
fit = Fit()
result_dm_2 = fit.run(datasets=[dataset_2])

In [ ]:
# Finally let's perform the test likelihood ratio between both models.

# Gammapy returns the -2 log(L)
logL_dm   = result_dm_2.optimize_result.total_stat

ts_dm_2 = (logL_nosrc - logL_dm)

print(ts_dm_2, np.sqrt(ts_dm_2)) ### test statistic is too low, <5. 

In [ ]:
# Again, We can check the result of the Fit:
result_dm_2.parameters.to_table()

OMG why do we get a negative value for scale? How we can handle this?
As we have not restricted the values where the fit can search in the whole space for the parameter... 
Let's learn two solutions

In [ ]:
# Gammapy has a way to access the likelihood profile as an array
# We need to define the number of entries and the minimum and maximum value:
# The profile method does only allow positive values for the scale parameter

dataset_2.models.parameters['scale'].scan_n_values = 100
dataset_2.models.parameters['scale'].scan_min = 10e-6
dataset_2.models.parameters['scale'].scan_max = 1000

# We are ready to get the likelihood profile
profile = fit.stat_profile(datasets=[dataset_2], parameter='scale', reoptimize=False)
# this method runs on the parameters one at a time, while keeping other params fixed
# if want to change then reoptimize=True

In [ ]:
total_stat  = dataset_2.stat_sum() # This is the likelihood of the fit -> (-2ln(L))
xvals = profile["P2gyc93K.spectral.scale_scan"]
y_profile = profile["stat_scan"]
yvals = profile["stat_scan"] - total_stat #stat_scan is the value of the likelihood in the maximum (minimum)

In [ ]:
# To reach to see our minimum, we add it manually to the array
xvals = np.insert(xvals, 0, result_dm_2.parameters['scale'].value)
yvals = np.insert(yvals, 0, 0)

In [ ]:
# Some useful values
min_xval = np.min(xvals)
max_xval = np.max(xvals)
min_yval = np.min(yvals)
max_yval = np.max(yvals)

In [ ]:
fig = plt.figure(figsize=(10, 5))

plt.plot(xvals,yvals)
plt.ylabel("Likelihood profile", fontsize=12)
plt.xlabel("Scale parameter", fontsize=12)
#plt.xlim(-50, 600)

plt.vlines(result_dm_2.parameters['scale'].value, 0, max_yval, ls="--", color='red')
plt.vlines(0, 0, max_yval, ls="--", color='red')

plt.show()

### Unbounded likelihood method

Let's use the real minimum value that we have, and search for the solution
We recall this will only be valid if the obtain limits is positive

In [ ]:
# Let's found the scale value corresponding to the 95% C.L 
# We don't want the corresponding negative value of the solution
# This corresponds to solve the next equation (one-sided distribution)
# Recall we only want to interpolate, son min and max values should be within the values that xvals conatins already

scale_found = brentq(interp1d(xvals, yvals-2.71, kind="quadratic"),
                    min_xval, max_xval,
                    maxiter=100,
                    rtol=1e-5,)

print('Check the scale parameter: ', scale_found)
sigma_v_ul = scale_found * DarkMatterAnnihilationSpectralModel.THERMAL_RELIC_CROSS_SECTION
sigma_v_ul = sigma_v_ul.value
print('!!!!!!!!!!!!!! Final Results!!!!!!!!!!')
print(channel_fit, mass_fit, scale_found, sigma_v_ul)

In [ ]:
fig = plt.figure(figsize=(10, 5))

plt.plot(xvals, yvals)
plt.ylabel("Likelihood profile", fontsize=12)
plt.xlabel("Scale parameter", fontsize=12)

# Line corresponding to 95% CL from minimum
plt.hlines(2.71, result_dm_2.parameters['scale'].value, max_xval, ls="--", color='red')
plt.hlines(0, result_dm_2.parameters['scale'].value, max_xval, ls="--", color='red')
plt.vlines(result_dm_2.parameters['scale'].value, 0, max_yval, ls="--", color='red')
plt.vlines(scale_found, 0, max_yval, ls="--", color='red')

#plt.ylim(-10, 100)

plt.show()

### Bounded likelihood method

We set the as best value scale = 0, and search the UL from this
We this will be always valid, and produce more conservative limits

In [ ]:
y_profile = profile["stat_scan"]
y_profile = np.insert(y_profile, 0, total_stat)

In [ ]:
profile_interp = interp1d(xvals, y_profile, kind="quadratic")

In [ ]:
# We need to reset the yvals to the value at scale = 0 instead of best_stat
yvals_rescaled = y_profile -  profile_interp(0)

In [ ]:
# Let's found the scale value corresponding to the 95% C.L 

scale_found_2 = brentq(interp1d(xvals, yvals_rescaled-2.71, kind="quadratic"),
                    min_xval, max_xval,
                    maxiter=100,
                    rtol=1e-5,)

print('Check the scale parameter: ', scale_found)
sigma_v_ul_2 = scale_found_2 * DarkMatterAnnihilationSpectralModel.THERMAL_RELIC_CROSS_SECTION
sigma_v_ul_2 = sigma_v_ul_2.value
print('!!!!!!!!!!!!!! Final Results!!!!!!!!!!')
print(channel_fit, mass_fit, scale_found_2, sigma_v_ul_2)

In [ ]:
fig = plt.figure(figsize=(10, 5))

plt.plot(xvals, yvals_rescaled)
plt.ylabel("Likelihood profile", fontsize=12)
plt.xlabel("Scale parameter", fontsize=12)

# Line corresponding to 95% CL from minimum
plt.hlines(2.71, result_dm_2.parameters['scale'].value, max_xval, ls="--", color='red')
plt.hlines(0, result_dm_2.parameters['scale'].value, max_xval, ls="--", color='red')
plt.vlines(0, 0, np.max(yvals_rescaled), ls="--", color='red')
plt.vlines(scale_found_2, 0, np.max(yvals_rescaled), ls="--", color='red')

#plt.ylim(-10, 100)

plt.show()